In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import joblib

# 예시 데이터
head = [
    "경제 성장률 상승으로 주가 상승세",
    "기업 이익 증가에 따른 투자자 신뢰 증가",
    "실업률 상승으로 경제 불안정 우려",
    "금리 인상으로 주식 시장에 악영향",
    "수출 감소로 인한 경제 성장 둔화",
    "신기술 개발로 기업 가치 상승 예상"
]
type = [1, 1, 0, 0, 0, 1]  # 호재는 1, 악재는 0

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(head)
y = type

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 훈련
model = LogisticRegression()
model.fit(X_train, y_train)

# 모델 평가
score = model.score(X_test, y_test)
print(f"정확도: {score:.2f}")

# 모델과 벡터라이저를 파일로 저장
joblib.dump(model, 'news_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')


정확도: 0.00


['tfidf_vectorizer.pkl']

In [ ]:
# 모델과 벡터라이저를 파일로부터 불러오기
model = joblib.load('news_model.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')

# 새로운 데이터에 대한 예측
new_news = ["금융 위기로 인한 경제 충격", "기술 혁신으로 새로운 시장 창출"]
new_news_vector = tfidf_vectorizer.transform(new_news)
predictions = model.predict(new_news_vector)
print(predictions)   # [0 1]과 같이 악재(0)와 호재(1)를 나타냄


In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="gabrielyang/finance_news_classifier-KR_v7")
text = "금리 인상으로 주식 시장에 악영향"
result = pipe(text)
print(result)

/Users/oasispapa/Develop/Homework/TextAnalyzer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'label': 'negative', 'score': 0.999362051486969}]


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("gabrielyang/finance_news_classifier-KR_v7")
model = AutoModelForSequenceClassification.from_pretrained("gabrielyang/finance_news_classifier-KR_v7")

In [3]:
# 호악재 분석 (고급 버전)
def analyze_news(news):
    # Load model directly
    tokenizer = AutoTokenizer.from_pretrained("gabrielyang/finance_news_classifier-KR_v7")
    model = AutoModelForSequenceClassification.from_pretrained("gabrielyang/finance_news_classifier-KR_v7")
    # Tokenize input text
    inputs = tokenizer(news, return_tensors="pt")
    # Perform inference
    outputs = model(**inputs)
    # Get predicted class
    predicted_class_idx = outputs.logits.argmax().item()
    # Get predicted class name
    predicted_class_name = model.config.id2label[predicted_class_idx]
    return predicted_class_name

In [4]:
text = "금리 인상으로 주식 시장에 악영향"
result = analyze_news(text)
print(result)  # 악재

negative


In [2]:
import requests
from bs4 import BeautifulSoup
import telegram
import asyncio
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline


# 연합뉴스 (오늘)
def get_news_from_yonhap(): 
    # URL of the RSS feed
    url = "https://www.yonhapnewstv.co.kr/category/news/economy/feed/"

    # Fetch the RSS feed
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses

    # Parse the XML
    soup = BeautifulSoup(response.text, "xml")
    # 뉴스 제목
    titles = [item.find('title').text for item in soup.find_all('item')]
    # 뉴스 내용
    encoded_contents = [item.find('content:encoded').text for item in soup.find_all('item')]
    # 링크
    links = [item.find('link').text for item in soup.find_all('item')]
    return titles, encoded_contents, links

In [5]:
title, news, links = get_news_from_yonhap()


In [8]:
text = news[0]

In [12]:
from transformers import pipeline

pipe = pipeline("text-classification", model="gabrielyang/finance_news_classifier-KR_v7")   

# 한번에 분석하기 힘든 긴 텍스트 때문에 청크로 자른다. 
max_length = 512  # Assuming the model's max token length is 512
text_chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]

# 예측을 진행하고 결과를 청크별로 나눠 담는다
results = []
for chunk in text_chunks:
    result = pipe(chunk)
    result[0]['length'] = len(chunk)
    results.append(result)

In [13]:
results

[[{'label': 'negative', 'score': 0.7202937602996826, 'length': 414}]]

In [14]:

weighted_results = {}
for result in results:
    for res in result:
        label = res['label']
        length = res['length']
        if label in weighted_results:
            weighted_results[label] += length
        else:
            weighted_results[label] = length

In [17]:
most_common_label = max(weighted_results, key=weighted_results.get)
most_common_label

'negative'